In [21]:
import cv2
import glob
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader


In [22]:
batch_size = 64
num_classes = 2
learning_rate = 0.001
num_epochs = 20
PATH = '/content/drive/MyDrive/Projects/simple cnn/'

In [39]:
class KafshDataset(Dataset):
    def __init__(self, data_path,  transform=None):
        self.data_path = data_path
        self.classes = glob.glob(data_path+'/*')
        self.transform = transform
        self.image_paths = []
        for c in self.classes:
          self.image_paths += glob.glob(c+'/*')
        self.label_map = {'giveh': 0, 'katooni': 1}

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_filepath = self.image_paths[idx]
        image = cv2.imread(image_filepath)
        image = cv2.resize(image, dsize=(128, 128), interpolation=cv2.INTER_CUBIC)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        label = image_filepath.split('/')[-2]
        tensor_label = torch.tensor(self.label_map[label])

        if self.transform:
            image = self.transform(image)

        return image, tensor_label

In [73]:
transform = transforms.ToTensor()

In [76]:
train_dataset = KafshDataset(PATH+"Kafsh/train", transform)
test_dataset = KafshDataset(PATH+"Kafsh/test", transform)

train_loader = DataLoader(
    dataset = train_dataset,
    batch_size = batch_size,
    shuffle = True)

test_loader = DataLoader(
    dataset = test_dataset,
    batch_size = batch_size,
    shuffle = False)

In [69]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(SimpleCNN, self).__init__()

        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )

        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )

        self.fc = nn.Sequential(
            nn.Linear(10092, 120),
            nn.ReLU(),
            nn.Linear(120, 60),
            nn.ReLU(),
            nn.Linear(60, num_classes)
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [77]:
model = SimpleCNN(num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [78]:
for epoch in range(num_epochs):
    correct = 0
    total = 0
    for i, (images, labels) in enumerate(train_loader):

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
    print('\t Accuracy of the network : {} %'.format(100 * correct / total))





Epoch [1/20], Loss: 0.6904
	 Accuracy of the network : 47.5 %
Epoch [2/20], Loss: 0.6742
	 Accuracy of the network : 50.0 %
Epoch [3/20], Loss: 0.6491
	 Accuracy of the network : 61.875 %
Epoch [4/20], Loss: 0.6117
	 Accuracy of the network : 72.5 %
Epoch [5/20], Loss: 0.5733
	 Accuracy of the network : 72.5 %
Epoch [6/20], Loss: 0.4601
	 Accuracy of the network : 77.5 %
Epoch [7/20], Loss: 0.3074
	 Accuracy of the network : 83.125 %
Epoch [8/20], Loss: 0.2868
	 Accuracy of the network : 86.875 %
Epoch [9/20], Loss: 0.1872
	 Accuracy of the network : 86.25 %
Epoch [10/20], Loss: 0.3732
	 Accuracy of the network : 86.25 %
Epoch [11/20], Loss: 0.3091
	 Accuracy of the network : 91.25 %
Epoch [12/20], Loss: 0.1808
	 Accuracy of the network : 89.375 %
Epoch [13/20], Loss: 0.2292
	 Accuracy of the network : 93.125 %
Epoch [14/20], Loss: 0.2096
	 Accuracy of the network : 93.125 %
Epoch [15/20], Loss: 0.2933
	 Accuracy of the network : 95.0 %
Epoch [16/20], Loss: 0.2588
	 Accuracy of the net

In [79]:
with torch.no_grad():
    correct = 0
    total = 0
    for i, (images, labels) in enumerate(test_loader):
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network : {} %'.format(100 * correct / total))




Accuracy of the network : 92.5 %


In [80]:
torch.save(model.state_dict(), PATH+'KafshModel.pth')